In [51]:
import numpy as np
import pandas as pd

In [52]:
DATASET_PATH = "../Bitcoin_4_1_2020-4_1_2022_historical_data_coinmarketcap.csv"

In [53]:
def load_btc_data(filepath: str, datetime_col='timeClose', price_col='close',
                  freq='D', split_date='2020-01-01'):
    """
    Load BTC OHLCV dataset and resample to daily frequency using last close price.
    Split into training and testing sets.

    Parameters:
    - filepath: Path to CSV file
    - datetime_col: Name of datetime column
    - price_col: Price column to use for strategy
    - freq: Resampling frequency (default 'D' = daily)
    - split_date: Cut-off date to split training/testing

    Returns:
    - train_df, test_df: DataFrames with Close prices
    """
    
    df = pd.read_csv(filepath, delimiter=';')
    print(df.info())

    df[datetime_col] = pd.to_datetime(df[datetime_col])
    df = df.set_index(datetime_col).sort_index()
    
    df_resampled = df[[price_col]].resample(freq).last().dropna()
    df_resampled.rename(columns={price_col: 'Close'}, inplace=True)
    
    train_df = df_resampled[df_resampled.index < split_date].copy()
    test_df = df_resampled[df_resampled.index >= split_date].copy()
    
    return train_df, test_df

In [54]:
def compute_ema_signal(df, fast_window, slow_window):
    """
    Generate EMA crossover signal: +1 (buy), -1 (sell), 0 (hold)
    """
    ema_fast = df['Close'].ewm(span=fast_window, adjust=False).mean()
    ema_slow = df['Close'].ewm(span=slow_window, adjust=False).mean()
    signal = np.where(ema_fast > ema_slow, 1, np.where(ema_fast < ema_slow, -1, 0))
    return pd.Series(signal, index=df.index)

def compute_rsi_signal(df, window, threshold):
    """
    Generate RSI-based signal: +1 (oversold), -1 (overbought), 0 (neutral)
    """
    delta = df['Close'].diff()
    up = delta.clip(lower=0).rolling(window=window).mean()
    down = -delta.clip(upper=0).rolling(window=window).mean()
    rs = up / (down + 1e-6)
    rsi = 100 - (100 / (1 + rs))

    signal = np.where(rsi < (100 - threshold), 1, np.where(rsi > threshold, -1, 0))
    return pd.Series(signal, index=df.index)

def compute_bb_signal(df, window, sigma):
    """
    Generate Bollinger Band signal: +1 (below lower band), -1 (above upper band), 0 (inside)
    """
    ma = df['Close'].rolling(window=window).mean()
    std = df['Close'].rolling(window=window).std()
    upper = ma + sigma * std
    lower = ma - sigma * std

    signal = np.where(df['Close'] > upper, -1, np.where(df['Close'] < lower, 1, 0))
    return pd.Series(signal, index=df.index)

In [55]:
def composite_strategy_signal(df, params):
    """
    Combine signals from EMA, RSI, BB using weighted average and threshold.

    Params:
    - params: [d1, d2, d3, rsi_thresh, d4, sigma, w1, w2, w3, decision_thresh]
    Returns:
    - Series of final trading signal (+1/-1/0)
    """
    d1, d2, d3, rsi_thresh, d4, sigma, w1, w2, w3, threshold = params
    s1 = compute_ema_signal(df, int(d1), int(d2))
    s2 = compute_rsi_signal(df, int(d3), rsi_thresh)
    s3 = compute_bb_signal(df, int(d4), sigma)

    weighted_sum = (w1 * s1 + w2 * s2 + w3 * s3) / (w1 + w2 + w3 + 1e-8)
    final_signal = np.where(weighted_sum > threshold, 1, np.where(weighted_sum < -threshold, -1, 0))
    return pd.Series(final_signal, index=df.index)

In [56]:
def backtest_with_fee(df, params, initial_cash=1000, fee_rate=0.03):
    """
    Simulate trading with fee, switching between cash and BTC only.
    
    Returns:
    - df with equity curve
    - final portfolio value (fitness score)
    """
    df = df.copy()
    df['Signal'] = composite_strategy_signal(df, params)
    
    cash = initial_cash
    btc = 0.0
    equity_list = []
    state = 'cash'

    for i in range(1, len(df)):
        price = df['Close'].iloc[i]
        signal = df['Signal'].iloc[i]

        if state == 'cash' and signal == 1:
            btc = (cash * (1 - fee_rate)) / price
            cash = 0.0
            state = 'btc'
        elif state == 'btc' and signal == -1:
            cash = btc * price * (1 - fee_rate)
            btc = 0.0
            state = 'cash'
        
        equity = cash if state == 'cash' else btc * price
        equity_list.append(equity)

    # final forced liquidation if in BTC
    if state == 'btc':
        final_price = df['Close'].iloc[-1]
        cash = btc * final_price * (1 - fee_rate)
        btc = 0.0
        equity = cash
    else:
        equity = cash

    df = df.iloc[1:]
    df['Equity'] = equity_list
    return df, equity

In [57]:
def fitness_function(params, df):
    """
    Evaluate final capital after backtesting using given parameters.
    """
    _, final_equity = backtest_with_fee(df, params)
    return final_equity

In [ ]:
def particle_swarm_optimizer(fitness_func, data, bounds, num_particles=10, max_iter=30, w=0.5, c1=1.5, c2=1.5):
    """
    Particle Swarm Optimization (PSO) for continuous parameter tuning.
    """

    # Initialising the particles
    dim = len(bounds)
    pos = np.random.rand(num_particles, dim)
    vel = np.zeros((num_particles, dim))
    for i in range(dim):
        pos[:, i] = bounds[i][0] + pos[:, i] * (bounds[i][1] - bounds[i][0])

    # Initialising the best positions
    pbest = pos.copy()
    pbest_val = np.array([fitness_func(p, data) for p in pos])
    gbest = pbest[np.argmax(pbest_val)].copy()
    gbest_val = max(pbest_val)


    # optimisation of the particles
    for t in range(max_iter):
        for i in range(num_particles):
            r1, r2 = np.random.rand(dim), np.random.rand(dim)
            vel[i] = w * vel[i] + c1 * r1 * (pbest[i] - pos[i]) + c2 * r2 * (gbest - pos[i])
            pos[i] += vel[i]
            pos[i] = np.clip(pos[i], [b[0] for b in bounds], [b[1] for b in bounds])
            val = fitness_func(pos[i], data)
            print(f" Particle Value: {val}")

            if val > pbest_val[i]:
                pbest[i] = pos[i]
                pbest_val[i] = val
                if val > gbest_val:
                    gbest = pos[i]
                    gbest_val = val

    # Return the best position and its value
    return gbest, gbest_val

In [59]:
train_df,test_df=load_btc_data(filepath=DATASET_PATH)

bounds = [
    (5, 30),     # d1: EMA fast
    (20, 90),    # d2: EMA slow
    (7, 28),     # d3: RSI window
    (60, 80),    # RSI threshold
    (10, 50),    # BB window
    (1.5, 3.0),  # BB std
    (0.0, 1.0),  # w1
    (0.0, 1.0),  # w2
    (0.0, 1.0),  # w3
    (0.2, 0.7)   # decision threshold
]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timeOpen   400 non-null    object 
 1   timeClose  400 non-null    object 
 2   timeHigh   400 non-null    object 
 3   timeLow    400 non-null    object 
 4   name       400 non-null    int64  
 5   open       400 non-null    float64
 6   high       400 non-null    float64
 7   low        400 non-null    float64
 8   close      400 non-null    float64
 9   volume     400 non-null    float64
 10  marketCap  400 non-null    float64
 11  timestamp  400 non-null    object 
dtypes: float64(6), int64(1), object(5)
memory usage: 37.6+ KB
None


In [60]:
# 🚀 Particle Swarm Algorithm
best_pso_params, best_pso_score = particle_swarm_optimizer(
    fitness_func=fitness_function,
    data=train_df,
    bounds=bounds,
    num_particles=20,
    max_iter=50
)

print(best_pso_params, best_pso_score)

 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle Value: 1000
 Particle 